In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/Titanic/train.csv'
test_path = '/content/drive/MyDrive/Titanic/test.csv'
training = pd.read_csv(train_path)
test = pd.read_csv(test_path)

training['traintest'] = 1
test['traintest'] = 0
test['Survived'] = np.nan
alldata = pd.concat([training, test])
%matplotlib inline
alldata.columns
training.shape
training.head(5)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
test.shape



In [ ]:
alldata.shape

In [ ]:
training.columns

In [ ]:
test.columns

In [ ]:
training.describe()

In [ ]:
training.describe().columns

In [ ]:
df_num = training[['Age','SibSp','Parch','Fare']]
df_cat = training[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [ ]:
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [ ]:
alldata

In [ ]:
print(df_num.corr())
sns.heatmap(df_num.corr())

In [ ]:
pd.pivot_table(training, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])


In [ ]:
for i in df_cat.columns:
    sns.barplot(x=df_cat[i].value_counts().index, y=df_cat[i].value_counts()).set_title(i)
    plt.show()

In [ ]:
print(pd.pivot_table(training, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Sex', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count'))


In [ ]:
training['cabin_multiple'] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
training['cabin_multiple'].value_counts()


In [ ]:
pd.pivot_table(training, index = 'Survived', columns = 'cabin_multiple', values = 'Ticket' ,aggfunc ='count')

In [ ]:
training['cabin_adv'] = training.Cabin.apply(lambda x: str(x)[0])
print(training.cabin_adv.value_counts())

In [ ]:
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
training['numeric_ticket'].value_counts()

In [ ]:
pd.set_option("display.max_rows", None)
print(training)

In [ ]:
training['ticket_letters'].value_counts()

In [ ]:
pd.pivot_table(training,index='Survived',columns='numeric_ticket', values = 'Ticket', aggfunc='count')

In [ ]:
pd.pivot_table(training,index='Survived',columns='ticket_letters', values = 'Ticket', aggfunc='count')

In [ ]:
training.Name.head(50)

In [ ]:
training['name_title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
training.name_title.head(50)

In [ ]:
training['name_title'].value_counts().index

In [ ]:
training['name_title'].value_counts()

In [ ]:
alldata['cabin_multiple'] = alldata.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
alldata['cabin_adv'] = alldata.Cabin.apply(lambda x: str(x)[0])
alldata['numeric_ticket'] = alldata.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
alldata['ticket_letters'] = alldata.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
alldata['name_title'] = alldata.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
alldata['Age']

In [ ]:
alldata.Age = alldata.Age.fillna(alldata.Age.median())
alldata.Fare = alldata.Fare.fillna(training.Fare.median())
alldata['Age']


In [ ]:
alldata.columns


In [ ]:
training.columns

In [ ]:
test.columns

In [ ]:
alldata.tail(10)

In [ ]:
alldata.dropna(subset=['Embarked'],inplace = True)

In [ ]:
alldata['norm_sibsp'] = np.log(alldata.SibSp+1)

In [ ]:
alldata['norm_sibsp'].hist()

In [ ]:
alldata['norm_fare'] = np.log(alldata.Fare+1)
alldata['norm_fare'].hist()

In [ ]:
alldata.Pclass = alldata.Pclass.astype(str)
alldata.info()

In [ ]:
alldata.columns

In [ ]:
alldata.dtypes

In [ ]:
numerical_col = alldata.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_col = alldata.select_dtypes(include=['object']).columns.tolist()

numerical_col


In [ ]:
all_dummies = pd.get_dummies(alldata[['Age','Pclass','Sex','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title','traintest']])

In [ ]:
all_dummies.shape

In [ ]:
all_dummies.columns


In [ ]:
all_dummies.traintest

In [ ]:
xtrain = all_dummies[all_dummies.traintest == 1].drop(['traintest'], axis =1)
xtest = all_dummies[all_dummies.traintest == 0].drop(['traintest'], axis =1)

In [ ]:
ytrain = alldata[alldata.traintest==1].Survived

In [ ]:
ytrain.shape

In [ ]:
all_dummies.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(all_dummies_scaled[['Age','SibSp','Parch','norm_fare']])
all_dummies_scaled

X_train_scaled = all_dummies_scaled[all_dummies_scaled.traintest == 1].drop(['traintest'], axis =1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.traintest == 0].drop(['traintest'], axis =1)

y_train = alldata[alldata.traintest==1].Survived

In [ ]:
all_dummies_scaled.columns

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
gnb = GaussianNB()
cv = cross_val_score(gnb,X_train_scaled,y_train,cv=10)
print(cv)
print(cv.mean())

In [ ]:
lr = LogisticRegression(max_iter = 5000)
cv = cross_val_score(lr,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
svc = SVC(probability = True)
cv = cross_val_score(svc,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
xgb = XGBClassifier(random_state =1)
cv = cross_val_score(xgb,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
# from sklearn.ensemble import VotingClassifier
# voting_clf = VotingClassifier(estimators = [('lr',lr),('knn',knn),('rf',rf),('gnb',gnb),('svc',svc),('xgb',xgb)], voting = 'soft')

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lr',lr),('knn',knn),('rf',rf),('gnb',gnb),('svc',svc),('xgb',xgb)], voting = 'hard')

In [ ]:
# cv = cross_val_score(voting_clf_s, X_train_scaled, y_train, cv=5)
# print(cv)
# print(cv.mean())

In [ ]:
cv = cross_val_score(voting_clf, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

In [ ]:
voting_clf.fit(X_train_scaled,y_train)
y_hat_base_vc = voting_clf.predict(X_test_scaled).astype(int)
basic_submission = {'PassengerId': test.PassengerId, 'Survived': y_hat_base_vc}
base_submission = pd.DataFrame(data=basic_submission)
base_submission.to_csv('base_submission.csv', index=False)